# Clustering trees to find cohesive plantings

Given the locations of all trees, we want to assign groups to the trees, which represent separate plantings. A requirement is, that arbitrarily-shaped groups of trees are found, i.e. circles, straight lines or randomly scattered trees are all recognised as a group, assuming they're close enough to each other.
This requirement can be fulfilled by density-based clustering methods, for which DBSCAN is the classical and proven apprach. One specific feature that is important in DBSCAN is that trees which are not assigned to clusters are assigned to a noise cluster with id -1. We are going to use the scikit-learn implementation.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from math import radians
from sklearn.metrics.pairwise import haversine_distances
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
import folium

## Clustering based on location data only

For the clustering to work, we need to find good parameter values that determine when points are grouped as a cluster. We have to choose two separate parameters. The first one is _eps_, which determines how close two treees have to be, to still be considered for possibly belonging in one cluster. The second parameter is _min_pts_, which, roughly spoken, indicates how many trees have to be close to each other to be considered a cluster. 

Since we don't know beforehand which trees belong together with which other trees, we can't just try different parameters and check how close the clustering is to the known solution in an automated way. Therefore, the most sensible thing to do is to simply manually inspect the results of clusterings. We will use the _folio_ package for this, to create an interactive map with markers on it for each tree. However, due to the large amount of trees involved, this map is absolutely not navigable for all of the trees. As a first measure, we can make the assumption that tree plantings generally don't cross country borders, and cluster each country's trees separately. If memory is an issue, this could also be done in the final solution. Otherwise, the efficient data structure used internally by the algorithm should already avoid that distances between trees in Canada and Tanzania are computed, so run time would not be the issue. Since Tanzania's 200k+ trees are still too many to display all of them on a map at the same time, I further filter the trees to have a latitude <= -5.0. We can't manually inspect all clusters anyway, so subsampling further in this way seems okay.



In [302]:
df = pd.read_csv("tree_data_cc_single_core.csv")
df["time_created"] = df["time_created"].astype("datetime64")
df["time_created"] = df["time_created"].map(lambda x: x.toordinal())

# Reduce data set to be able to 
# 1. perform clustering on bad computer and 
# 2. not have the map be laggy later on
df = df[df.country_code == "TZ"]
df = df[df["lat"] <= -5.0].reset_index(drop=True)

X = df[["lat", "lon"]]
X_rad = X.applymap(radians)

In [3]:
# Given the maximum distance in meters between trees in same cluster, compute eps value for clustering
def compute_eps(max_dist=80):
    earth_radius = 6371000
    return max_dist/earth_radius

In [4]:
# Precompute sparse distance matrix
# This will be needed to further filter which trees we draw on the map later on

eps = compute_eps(80)
nn = NearestNeighbors(n_neighbors=2, radius=eps, metric="haversine", n_jobs=-1, algorithm="ball_tree")
%time nn.fit(X_rad)

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 165 ms


NearestNeighbors(algorithm='ball_tree', metric='haversine', n_jobs=-1,
                 n_neighbors=2, radius=1.2556898446083818e-05)

In [5]:
%time dist_mat = nn.radius_neighbors_graph(mode="distance", sort_results=True)

CPU times: user 26.3 s, sys: 45.7 s, total: 1min 11s
Wall time: 34.3 s


In [6]:
from scipy.sparse import save_npz
save_npz("dist_mat_tz_leqneg5_80m.npz", dist_mat)

In [7]:
# Compute clustering based on precomputed distance matrix
%time db = DBSCAN(eps = eps, min_samples = 2, metric="precomputed").fit(dist_mat)
labels = db.labels_

CPU times: user 952 ms, sys: 0 ns, total: 952 ms
Wall time: 1.39 s


In [8]:
labels_df = pd.DataFrame(labels)
labels_df.to_csv("labels_tz_leqneg5_80m.csv", index=False)
del labels_df

In [12]:
# Compute label statistics
unique_labels = list(set(labels))
unique_labels.sort()

from collections import Counter
counter = Counter(labels)
print(counter)
print(f"Number of clusters: {len(unique_labels)}")

Counter({1: 3458, 0: 1425, 27: 1306, 24: 962, 25: 681, 13: 612, 26: 529, 11: 368, 12: 350, 18: 261, 9: 232, 10: 219, 5: 177, 15: 144, 29: 121, 16: 113, 17: 69, 8: 64, 21: 64, 7: 54, 4: 30, 2: 16, -1: 16, 19: 14, 14: 12, 23: 9, 6: 4, 3: 2, 20: 2, 22: 2, 28: 2})
Number of clusters: 31


# Creating an interactive map

For visual inspection, we want a map to look at, zoom in and out, to determine whether the clustering seems sensible, or parameters need to be changed. The _folium_ package allows us to do this in a very easy manner. Each tree is represented by a pin marker, to which we can assign different colors to easily differentiate different clusters. A blue marker in a bunch of red markers would be something we'd like to investigate, to make sure that the cluster assignment didn't go wrong there, because of suboptimal parameter choices.

In [11]:
# Preparation: Mapping cluster_id to a pin color
import folium

marker_colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
]

color_map = dict({-1:'black'})
for label in unique_labels:
    if label!=-1:
        color_map[label] = marker_colors[label%len(marker_colors)]

## Filtering which trees to draw

Since the map becomes _very_ slow and unresponsive when drawing most of Tanzania's trees on it, we further filter which trees to draw. The general idea is that we are interested in the broad shape of a cluster, to see whether its close to another cluster or not. We don't need to know that five trees are within 10m of each other when looking at the map, so we remove trees in each cluster that are very close to each other.

For each cluster, we pick one starting tree. All trees of the same cluster, which are closer to the starting tree than _eps_draw_, are filtered out. We repeat this process for all trees in the cluster, which have not yet been filtered out. In this way, we hope to see more of the spatial expansion of the cluster, instead of the local clumps where many trees are close to each other. This method isn't perfect, but worked fine to make the map useable. 

This method is why we need to pre-compute the distance matrix before running DBSCAN, since the DBSCAN method does not return its distance matrix. 

In [14]:
eps_draw = compute_eps(70) # Only draw trees that have a minimum distance of 70m from current tree
cluster_points_to_draw = dict() # cluster_id -> {points}

for cluster_id in unique_labels:
    # check_point indicates the indices of trees which should be checked
    # If a tree is too close to the current point, it will be marked here, so that it won't be checked at all.
    check_point = labels==cluster_id
    # draw_points indicates the indices of trees that will be drawn on the map
    draw_points = np.zeros(shape=(len(labels)), dtype=bool)
    for current_point in range(len(check_point)):
        if not check_point[current_point]:
            continue
        draw_points[current_point] = True
        
        # Check all neighbors of the current tree. 
        # If they are far enough away, add them to the trees that will be drawn on the map
        sub_mat = dist_mat[current_point]
        for neighbor_id, v in zip(sub_mat.indices, sub_mat.data):
            # current_point increases strictly monotonically. 
            # Therefore, we have already checked lower neighbor_ids and can save time by not repeating that here
            if neighbor_id > current_point:
                # Only draw points on the map that are relatively far from already-drawn points
                if v >= eps_draw:
                    draw_points[neighbor_id] = True
                else:
                    check_point[neighbor_id] = False
                    
    cluster_points_to_draw[cluster_id] = np.nonzero(draw_points)[0]

n_pts_unfiltered = X.shape[0]
n_pts_filtered = 0
for l in unique_labels:
    n_pts_filtered += len(cluster_points_to_draw[l])
print(f"After filtering, we still draw {n_pts_filtered}/{n_pts_unfiltered} points. This is a reduction of {(1-(n_pts_filtered/n_pts_unfiltered))*100} %.")

After filtering, we draw 3870/11318 points still being drawn. This is a reduction of 65.80667962537551 %.


In [15]:
# Draw map with filtered trees
x0, y0 = X.iloc[0]
m = folium.Map(
    location=[x0, y0],
    zoom_start=5,
    tiles='openstreetmap'
)

for cluster_id in unique_labels:
    for point_id in cluster_points_to_draw[cluster_id]:
        x, y = X.iloc[point_id]
        folium.Marker([x,y], tooltip=str(cluster_id), icon=folium.Icon(color=color_map[cluster_id])).add_to(m)

In [17]:
m.save('map_TZ_leqneg5_80m.html')

In [27]:
# Shows map in notebook (slow, because the map contains a lot of markers)
# Alternatively, you can open the html file in a browser

# If you only want to get an impression of how this map looks, the one three cells further down works a lot nicer.
m

## Closely investigating specific clusters

After manually investigating some of the clusters, we may want to investigate some potentially problematic clusters more thoroughly. For this, we want to look at all of the trees they contain, without filtering any, as we have done before. 

In [18]:
# Draw map fully, but for only certain clusters
x0, y0 = X.iloc[0]
m = folium.Map(
    location=[x0, y0],
    zoom_start=5,
    tiles='openstreetmap'
)

for cluster_id in [-1,2,3,18,19,21,27]:
    for _, (lat, lon) in X[labels==cluster_id].iterrows():
        folium.Marker([lat, lon], tooltip=str(cluster_id), icon=folium.Icon(color=color_map[cluster_id])).add_to(m)

In [19]:
m

## Parameter choices for spatial clustering

_min_samples_ should be 1 or 2.

_min_samples_=1 means that a single point is always treated as a full cluster, which eliminates the noise cluster with _cluster_id_ -1. This noise cluster normally catches all points which are not assigned to a proper cluster consisting of multiple points.

_min_samples_=2 means that a cluster needs to contain two points to be considered a distinct cluster. Points which are too isolated for that to occur are assigned to the noise cluster.

For any values of _min_samples_>=3, there will be cases in which single trees will be left out, since they are too far from a group of trees. For example, the last tree at the end of a line of equidistant trees might not be included in the cluster containing the other lined-up trees.

_eps_ was chosen based on all trees in Thailand and the subset of trees in Tanzania chosen above. It should be bigger than 50m, based on the large batch of trees in Thailand. 60m was enough for that example.
Judging from a location in eastern Dar es Salaam (TZ), 60m was not enough to result in intuitive clusters, while 80m was. All other clusters seemed reasonable, too in this setting.
Therefore, we choose _eps_ to represent a distance of 80m.

It might make sense to investigate further areas to validate this parameter choice.

# Clustering based on location AND time

Since the location-based clustering works, we will now move to adding the time component. 

We have agreed that a simple hard boundary of 3 months should be enough to distinguish between different plantings. If two trees are planted more than 90 days apart, they will be considered to not belong to one cluster. This is, of course, a strong simplication, but any more nuanced approach will have to balance a local distance with a temporal distance, i.e.: If two trees are 20m apart and were planted with a temporal lag of 50 days, how much closer should they be considered than two trees that are 35m apart, but planted within three days? 

We first want to look at how coherent the clusters that we found are, in terms of planting time. 

# Analyse planting time

In [303]:
for cluster_id in unique_labels:
    # Trees in the noise cluster aren't associated with each other
    if cluster_id == -1:
        continue
    dates = df[labels==cluster_id]["time_created"]
    print(f"cluster_id: {cluster_id}, time span within cluster: {dates.max()-dates.min()} days")


cluster_id: 0, time span within cluster: 39 days
cluster_id: 1, time span within cluster: 30 days
cluster_id: 2, time span within cluster: 0 days
cluster_id: 3, time span within cluster: 0 days
cluster_id: 4, time span within cluster: 212 days
cluster_id: 5, time span within cluster: 0 days
cluster_id: 6, time span within cluster: 0 days
cluster_id: 7, time span within cluster: 1 days
cluster_id: 8, time span within cluster: 0 days
cluster_id: 9, time span within cluster: 30 days
cluster_id: 10, time span within cluster: 0 days
cluster_id: 11, time span within cluster: 1 days
cluster_id: 12, time span within cluster: 9 days
cluster_id: 13, time span within cluster: 0 days
cluster_id: 14, time span within cluster: 0 days
cluster_id: 15, time span within cluster: 0 days
cluster_id: 16, time span within cluster: 0 days
cluster_id: 17, time span within cluster: 0 days
cluster_id: 18, time span within cluster: 0 days
cluster_id: 19, time span within cluster: 0 days
cluster_id: 20, time span

In [304]:
# Closer look at cluster 4, which has a large discrepancy between min and max date
from datetime import date
dates = df[labels==4]["time_created"].map(lambda x: date.fromordinal(x))
dates = dates.astype("datetime64")
df.groupby([dates.dt.year, dates.dt.month]).count()

,,id,time_created,lat,lon,country_code
time_created,time_created,,,,,
2019.0,8.0,27,27,27,27,27
2020.0,3.0,3,3,3,3,3


-> Two separate plantings in one cluster

In [239]:
# Closer look at cluster 0, which also has a larger spread of dates 
dates = df[labels==0]["time_created"].map(lambda x: date.fromordinal(x))
dates = dates.astype("datetime64")
df.groupby([dates.dt.year, dates.dt.month, dates.dt.day]).count()

id  time_created  lat  lon  \
time_created time_created time_created                                
2019.0       8.0          7.0             4             4    4    4   
                          11.0            1             1    1    1   
                          13.0            2             2    2    2   
                          15.0          506           506  506  506   
                          20.0          229           229  229  229   
             9.0          6.0             1             1    1    1   
                          7.0           109           109  109  109   
                          8.0           154           154  154  154   
                          9.0           170           170  170  170   
                          11.0            1             1    1    1   
                          13.0          245           245  245  245   
                          15.0            3             3    3    3   

                                        country_code  
time_created time_created time_created                
2019.0       8.0          7.0                      4  
                          11.0                     1  
                          13.0                     2  
                          15.0                   506  
                          20.0                   229  
             9.0          6.0                      1  
                          7.0                    109  
                          8.0                    154  
                          9.0                    170  
                          11.0                     1  
                          13.0                   245  
                          15.0                     3

-> Looks like a continuous planting effort, distributed over one month

## Extending the distance function with time-based distance

The simple approach of setting a hard temporal limit for how close trees can be to be considered as neighbors can be implemented as a simple distance function. We just have to extend the haversine distance function, which we used above, with a temporal distance, and return a very large distance if trees have been planted with too much temporal distance.

In [305]:
# If two trees have been planted more than 90 days apart, 
# we return a huge distance, which should lead them to not be considered in one cluster
# Otherwise, the distance is the haversine distance, as before.
   
def haversine_and_ordinal_time_dist(tree1, tree2):
    dist_t = abs(tree1[2] - tree2[2])
    if dist_t > 90:
        return 10000.0
    else:
        return haversine_distances([tree1[:2]], [tree2[:2]])[0,0]

## Problem

Upon further research, it turns out this distance function should not be used for the optimized version of DBSCAN that we use. Internally, DBSCAN uses a Ball-Tree data structure to make the neighborhood search very efficient. For this, the distance function needs to fulfill the triangle inequality (https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), which our simple function does not. For example, if we have three trees x,y,z, with the temporal distances td(x,y) = 60, td(y,z)=60, td(x,z)=100, then the above distance function would assign a distance of 10000 to the pair of trees (x,z), even though it would assign much less to the other two pairs. Therefore, the triangle unequality does not hold for this function. Therefore, we can not use our naiive approach.

## Alternative solution

One way to still consider the time is to perform purely spatial clustering first, ignoring time data, and then splitting clusters based on time data. E.g. if a cluster contains two distinct time spans in which trees were planted, then we'd create two new clusters from it. 

The failcases for this would be situations in which new trees are added to a cluster of old trees, but in locations which would not have been considered as a single cluster during normal clustering. For example, if we imagine trees being planted as a square, and only the four corner trees are replaced, they would normally not considered to belong to one cluster. I believe that in our use case it does make sense to do so, since they seem to represent a coherent effort, based on them being planted in the same existing cluster within the same time frame. With this perspective, what seemed like a failcase at first is actually a feature we might want our algorithm to have.

## Splitting clusters based on planting time

Assuming that we use the alternative solution outlined above, we now have spatially-clustered data. For each of those clusters, we want to sub-divide the cluster into subclusters of data that are separated by at least 90 days of temporal difference, thereby finding different plantings within a cluster. 

For each cluster: Sort the trees according to timestamps. Create subsequent groups of trees, cutting off when there's a gap of 90+ days between subsequent trees.

Note: This approach can lead to trees being included in one subcluster that are more 90 days apart. This can only occur, though, if there are other trees included, which bridge the temporal gap. E.g. if five trees are planted, each one 30 days after the previous one, then the first and the last will be 120 days apart, but still in one subcluster.

In [306]:
df["cluster"] = labels
df["subcluster"] = 0
df.sort_values("time_created", ascending=True, inplace=True)

for cluster_id in unique_labels:
    if cluster_id == -1:
        continue
        
    idx_list = []
    subcluster_assignment = []

    current_subcluster = 0
    prev_time = None
    
    for row in df[df["cluster"]==cluster_id].itertuples():
        current_time = row.time_created
        row_idx = row.Index
        if prev_time is None:
            prev_time = current_time
        elif (current_time - prev_time) > 90:
            current_subcluster += 1
            prev_time = current_time
        subcluster_assignment.append(current_subcluster)
        idx_list.append(row_idx) 

    df.loc[idx_list, "subcluster"] = subcluster_assignment


In [ ]:
df.to_csv("cluster_assignments_tz_leqneg5_80m.csv", index=False)

# Assigning new points after initial clustering

After computing an initial clustering using DBSCAN, new trees will be added to the system over time, as more trees are planted. We now want to think about how we can assign these new trees to clusters, without running everything above again on the updated state of the data base.

The method suggested [here](https://stackoverflow.com/questions/27822752/scikit-learn-predicting-new-points-with-dbscan) looks at each tree in the database, in the worst case, and computes the distance to it, until a tree is found that is close enough, so we can assign our tree to that cluster. This is repeated for each new data point. In the long run it would be preferable to somehow make use of the optimized data structures used by DBSCAN interntally. I currently don't know if it is even possible to get access to those in sklearn.

With a growing number of trees in the data base, the number of comparisons done for new points grows over time, so the runtime required will also grow. A pre-filtering by country is surely advisable here. If using the optimized data structures like ball-tree or KD-tree is necessary will have to be seen in practice. 

For now, we use the method linked above.

In [1]:
# Assigns new points to existing clusters (but not subclusters)
# Source: https://stackoverflow.com/a/35458920, pay attention to license
def dbscan_predict_new_points(XY_old, X_new, eps, metric=haversine_and_ordinal_time_dist):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find an old sample closer than EPS
        for xy_old in XY_old.itertuples(index=False):
            x_old = xy_old[:3]
            if metric(x_new, x_old) < eps:
                # Assign label of old_tree to x_new
                y_new[j] = xy_old.cluster
                break

    return y_new

# Assigns new points to subclusters.
# XY_old is a pandas dataframe, XY_new is a numpy array
def dbscan_predict_new_subclusters(XY_old, XY_new):
    Z_new = []
    for xy_new in XY_new:
        cluster_id = xy_new[3]
        time_created_new = xy_new[2]
        subcluster_id = None
        
        XY_old_subset = XY_old[XY_old["cluster"] == cluster_id]
        
        # If we find a tree with a subcluster that is close enough to the new tree, 
        # we can simply put it in there
        for xy_old in XY_old_subset.itertuples(index=False):
            if abs(time_created_new - xy_old.time_created) < 90:
                subcluster_id = xy_old.subcluster
                break
        # Otherwise, we create a new subcluster for it 
        if subcluster_id is None:
            subcluster_id = XY_old_subset.subcluster.max() + 1
        
        # Add the tree we just took care of to the existing dataframe,
        # since subsequent new trees might be in a cluster with it, instead of with old trees
        new_entry = np.append(xy_new, subcluster_id).tolist()
        Z_new.append(subcluster_id)
        XY_old = XY_old.append(new_entry,ignore_index=True)
        
    return XY_old, Z_new

NameError: name 'haversine_and_ordinal_time_dist' is not defined

In [315]:
# Test the method for assigning cluster to new points by computing the clusters for random existing points, 
# for which we know what the results should be
XY_old = df[["lat", "lon", "time_created", "cluster"]]
df_test_sample = df.sample(n=10)
X_new = df_test_sample[["lat", "lon", "time_created"]].to_numpy()
Y_new = dbscan_predict_new_points(XY_old, X_new, eps)
print(f"Assigned same clusters as ground truth:\n {df_test_sample['cluster'] == Y_new}")

Assigned same clusters as ground truth:
 8115    True
6211    True
4027    True
8760    True
7517    True
899     True
1171    True
9147    True
6541    True
8912    True
Name: cluster, dtype: bool


In [310]:
Y_new_exp = np.expand_dims(Y_new,-1)
XY_new = np.hstack([X_new,Y_new_exp])
XYZ_old = df[["lat", "lon", "time_created", "cluster", "subcluster"]]

In [314]:
# Test the method for assigning subclusters to new points
XYZ_updated, Z_new = dbscan_predict_new_subclusters(XYZ_old, XY_new)
print(f"Assigned same clusters as ground truth:\n {df_test_sample['subcluster'] == Z_new}")

Assigned same clusters as ground truth:
 7496     True
4115     True
10243    True
2312     True
9609     True
10175    True
2714     True
7728     True
9096     True
5920     True
Name: subcluster, dtype: bool
